In [1]:
#Import Libraries
import os
import json
import csv
import matplotlib.pyplot as plt
import sklearn
import tensorflow
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
import re
import torch
import torch.nn as nn
import time
import spacy
import kagglehub

#NLP Packages
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
from nltk.corpus import stopwords
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup
from IPython.display import display, clear_output

import pyarrow
import fastparquet
import dask.dataframe as dd

2025-03-02 19:13:43.057072: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-02 19:13:43.075704: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740960823.118107  280717 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740960823.131127  280717 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 19:13:43.178017: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
n_part = 1
file_count=20

df_dask = None

for i in range(min(n_part, file_count)):
    read_df = dd.read_parquet(os.path.join('data/lemm_stop_reviews_parquet', f"output_part_{i}.parquet"), engine="pyarrow")
    df_dask = read_df if i == 0 else dd.concat([df_dask, read_df])
# else:
    # df_dask = dd.read_parquet(os.path.join('data/lemm_stop_reviews_parquet', f"output_part_{i}.parquet"), engine="pyarrow")

all_df = df_dask.compute()

In [7]:
all_df

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,decide eat aware go take hour beginning end tr...,2018-07-07 22:09:11,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Restaurants, Breakfast & Brunch, Food, Americ...","{'Friday': '7:30-15:0', 'Monday': '7:30-15:0',..."
1,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,family diner buffet eclectic assortment large ...,2014-02-05 20:30:30,Kettle Restaurant,...,AZ,85713,32.207233,-110.980864,3.5,47,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Restaurants, Breakfast & Brunch]",None
2,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,wow yummy different delicious favorite lamb cu...,2015-01-04 00:01:03,Zaika,...,PA,19114,40.079848,-75.025080,4.0,181,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...",[Restaurants],"{'Friday': '11:0-22:0', 'Monday': None, 'Satur..."
3,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,cute interior owner give we tour upcoming pati...,2017-01-14 20:54:15,Melt,...,LA,70119,29.962102,-90.087958,4.0,32,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Sandwiches, Beer, Wine & Spirits, Bars, Food,...","{'Friday': '11:0-17:0', 'Monday': '0:0-0:0', '..."
4,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,long term frequent customer establishment go o...,2015-09-23 23:10:31,Dmitri's,...,PA,19147,39.938013,-75.148131,4.0,273,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Restaurants, Seafood]","{'Friday': '17:30-22:0', 'Monday': None, 'Satu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269657,6OyvyOBptbi1q08-e3mfsw,d-0QoPpdihk9IL-pAyr2UA,OAHRMd971k-yDQwRxzgc2g,2,3,1,0,excited finally check place holy hog bbq good ...,2016-09-05 21:11:51,4 Rivers Smokehouse,...,FL,33609,27.937719,-82.493507,4.0,343,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Food, Event Planning & Services, Southern, Re...","{'Friday': '11:0-20:0', 'Monday': '11:0-20:0',..."
269658,HKW8jTP-jizTtGlow18BlA,51SEYp4QKr4OCRnP6sdvjA,Cpl1-0iX6N7yauiJKhpXfA,5,2,0,0,understand negative review place order since o...,2018-08-30 17:48:48,Hong Kong Star Noodle House,...,PA,19020,40.111675,-74.955319,3.5,50,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...",[Restaurants],"{'Friday': '11:0-23:59', 'Monday': '11:0-22:0'..."
269659,R_v2thJfO0BHT1nCI8zW2g,bq1Si40ayzQedbsu9S70uQ,_aKr7POnacW_VizRKBpCiA,5,3,1,4,get well ice cold beer good sandwich town live...,2019-08-19 05:40:17,Blues City Deli,...,MO,63104,38.605024,-90.218110,5.0,991,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Bars, Restaurants, Nightlife, American (Tradi...","{'Friday': '10:30-15:0', 'Monday': '0:0-0:0', ..."
269660,8cdL1q7Q67pgSFFgjbckjg,Q1yOKVZO51McRHMpWhWr_Q,7fuFAkD_NqV5PQ7cA0Yzlg,5,0,0,0,long time fan sugar instead traditional shavin...,2018-09-06 22:45:36,Angelica Beauty and Waxing,...,FL,33629,27.923038,-82.493016,4.5,42,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...",[Beauty & Spas],"{'Friday': '9:0-20:0', 'Monday': '0:0-0:0', 'S..."


In [6]:
all_df['is_open'].describe()

count    269662.000000
mean          0.783032
std           0.412181
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: is_open, dtype: float64

In [8]:
def calculate_return_guest_percentage(reviews_business_df):
    df = reviews_business_df  # Ensure it has columns: 'user_id', 'business_id', 'text'
    
    # Count total unique guests per business
    total_guests = df.groupby('business_id')['user_id'].nunique().reset_index()
    total_guests = total_guests.rename(columns={'user_id': 'total_guest_count'})

    # Count the number of reviews per user per business
    repeat_visits = df.groupby(['business_id', 'user_id']).size().reset_index(name='review_count')

    # Filter businesses where a user has more than one review (return guests)
    repeat_visits = repeat_visits[repeat_visits['review_count'] > 1]

    # Count return guests per business
    return_guests = repeat_visits.groupby('business_id')['user_id'].nunique().reset_index()
    return_guests = return_guests.rename(columns={'user_id': 'return_guest_count'})

    # Merge return guests count with total guests count
    result = total_guests.merge(return_guests, on='business_id', how='left').fillna(0)

    # Calculate return guest percentage
    result['return_percentage'] = (result['return_guest_count'] / result['total_guest_count']) * 100
    result['return_percentage'] = result['return_percentage'].fillna(0)  # Replace NaN with 0%

    return result


In [11]:
return_guest_percentage = calculate_return_guest_percentage(all_df)

# add return_guest_percentage to all_df
all_df = all_df.merge(return_guest_percentage, on='business_id', how='left')

all_df

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,longitude,stars_y,review_count,is_open,attributes,categories,hours,total_guest_count,return_guest_count,return_percentage
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,decide eat aware go take hour beginning end tr...,2018-07-07 22:09:11,Turning Point of North Wales,...,-75.223639,3.0,169,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Restaurants, Breakfast & Brunch, Food, Americ...","{'Friday': '7:30-15:0', 'Monday': '7:30-15:0',...",69,3.0,4.347826
1,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,family diner buffet eclectic assortment large ...,2014-02-05 20:30:30,Kettle Restaurant,...,-110.980864,3.5,47,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Restaurants, Breakfast & Brunch]",None,19,0.0,0.000000
2,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,wow yummy different delicious favorite lamb cu...,2015-01-04 00:01:03,Zaika,...,-75.025080,4.0,181,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...",[Restaurants],"{'Friday': '11:0-22:0', 'Monday': None, 'Satur...",101,2.0,1.980198
3,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,cute interior owner give we tour upcoming pati...,2017-01-14 20:54:15,Melt,...,-90.087958,4.0,32,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Sandwiches, Beer, Wine & Spirits, Bars, Food,...","{'Friday': '11:0-17:0', 'Monday': '0:0-0:0', '...",26,3.0,11.538462
4,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1,1,2,1,long term frequent customer establishment go o...,2015-09-23 23:10:31,Dmitri's,...,-75.148131,4.0,273,0,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Restaurants, Seafood]","{'Friday': '17:30-22:0', 'Monday': None, 'Satu...",180,5.0,2.777778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269657,6OyvyOBptbi1q08-e3mfsw,d-0QoPpdihk9IL-pAyr2UA,OAHRMd971k-yDQwRxzgc2g,2,3,1,0,excited finally check place holy hog bbq good ...,2016-09-05 21:11:51,4 Rivers Smokehouse,...,-82.493507,4.0,343,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Food, Event Planning & Services, Southern, Re...","{'Friday': '11:0-20:0', 'Monday': '11:0-20:0',...",152,5.0,3.289474
269658,HKW8jTP-jizTtGlow18BlA,51SEYp4QKr4OCRnP6sdvjA,Cpl1-0iX6N7yauiJKhpXfA,5,2,0,0,understand negative review place order since o...,2018-08-30 17:48:48,Hong Kong Star Noodle House,...,-74.955319,3.5,50,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...",[Restaurants],"{'Friday': '11:0-23:59', 'Monday': '11:0-22:0'...",19,0.0,0.000000
269659,R_v2thJfO0BHT1nCI8zW2g,bq1Si40ayzQedbsu9S70uQ,_aKr7POnacW_VizRKBpCiA,5,3,1,4,get well ice cold beer good sandwich town live...,2019-08-19 05:40:17,Blues City Deli,...,-90.218110,5.0,991,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","[Bars, Restaurants, Nightlife, American (Tradi...","{'Friday': '10:30-15:0', 'Monday': '0:0-0:0', ...",587,16.0,2.725724
269660,8cdL1q7Q67pgSFFgjbckjg,Q1yOKVZO51McRHMpWhWr_Q,7fuFAkD_NqV5PQ7cA0Yzlg,5,0,0,0,long time fan sugar instead traditional shavin...,2018-09-06 22:45:36,Angelica Beauty and Waxing,...,-82.493016,4.5,42,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...",[Beauty & Spas],"{'Friday': '9:0-20:0', 'Monday': '0:0-0:0', 'S...",14,0.0,0.000000


In [ ]:
# import spacy
# os.system("./venv/bin/python -m spacy download en_core_web_sm")
# def extract_nouns(text):
#     nlp = spacy.load("en_core_web_sm")
#     doc = nlp(text)
#     nouns = [token.text for token in doc if token.pos_ == "NOUN"]
#     return nouns

# text = "The quick brown fox jumps over the lazy dog."
# nouns_only = extract_nouns(text)
# print(nouns_only)
# # Expected output: ['fox', 'dog']
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('universal_tagset')

def get_nouns(text):
    tokenized = pos_tag(word_tokenize(text), tagset='universal')
    # print(tokenized)
    nouns = [word for word, pos in tokenized if pos == 'NOUN']
    return ' '.join(nouns)

def get_adj(text):
    tokenized = pos_tag(word_tokenize(text), tagset='universal')
    # print(tokenized)
    nouns = [word for word, pos in tokenized if pos == 'ADJ']
    return ' '.join(nouns)


In [ ]:
print(get_nouns("The tamale has a hat"))

In [ ]:
the_text = all_df['text'][1]
# print(type(the_text))
print(the_text)
print(get_adj(the_text))

In [ ]:
the_text.split()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

def tfidf_summary(text, num_sentences=3):
    sentences = sent_tokenize(text)
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(sentences)
    
    sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
    top_sentence_indices = sentence_scores.argsort()[-num_sentences:][::-1]
    
    summary = [sentences[i] for i in sorted(top_sentence_indices)]
    return " ".join(summary)

text = "Your long text here..."
print(tfidf_summary(text, 2))

In [ ]:
text = """
Artificial Intelligence (AI) has transformed nearly every aspect of human life. Over the past few decades, AI has evolved from a theoretical concept into a practical tool that powers industries, automates tasks, and even influences human decision-making. From recommendation algorithms on streaming platforms to autonomous vehicles and medical diagnostics, AI has permeated various sectors, reshaping how we work, communicate, and live.

One of the most significant applications of AI is in healthcare. Machine learning algorithms analyze vast amounts of patient data to detect patterns and predict diseases earlier than traditional diagnostic methods. AI-driven robots assist in surgeries, ensuring precision and reducing recovery time for patients. Moreover, AI chatbots and virtual assistants help patients manage their health by providing immediate responses to medical inquiries.

Beyond healthcare, AI has revolutionized the business sector. Companies leverage AI for customer service automation, fraud detection, and personalized marketing. Chatbots powered by natural language processing (NLP) handle customer queries efficiently, reducing the need for human intervention. Businesses also use AI-driven analytics to predict consumer behavior, helping them make data-driven decisions that enhance profitability.

However, the widespread adoption of AI raises ethical concerns. One major issue is bias in AI algorithms, which can lead to unfair treatment in hiring, lending, and law enforcement. Since AI models learn from historical data, they can inherit societal biases, making it essential for researchers to develop fair and transparent AI systems. Additionally, the increasing reliance on AI for automation has sparked debates about job displacement, as machines replace human workers in various industries.
"""
print(tfidf_summary(text, 3))

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

def textrank_summary(text, num_sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, num_sentences)
    return " ".join(str(sentence) for sentence in summary)

text = """
Artificial Intelligence (AI) has transformed nearly every aspect of human life. Over the past few decades, AI has evolved from a theoretical concept into a practical tool that powers industries, automates tasks, and even influences human decision-making. From recommendation algorithms on streaming platforms to autonomous vehicles and medical diagnostics, AI has permeated various sectors, reshaping how we work, communicate, and live.

One of the most significant applications of AI is in healthcare. Machine learning algorithms analyze vast amounts of patient data to detect patterns and predict diseases earlier than traditional diagnostic methods. AI-driven robots assist in surgeries, ensuring precision and reducing recovery time for patients. Moreover, AI chatbots and virtual assistants help patients manage their health by providing immediate responses to medical inquiries.

Beyond healthcare, AI has revolutionized the business sector. Companies leverage AI for customer service automation, fraud detection, and personalized marketing. Chatbots powered by natural language processing (NLP) handle customer queries efficiently, reducing the need for human intervention. Businesses also use AI-driven analytics to predict consumer behavior, helping them make data-driven decisions that enhance profitability.

However, the widespread adoption of AI raises ethical concerns. One major issue is bias in AI algorithms, which can lead to unfair treatment in hiring, lending, and law enforcement. Since AI models learn from historical data, they can inherit societal biases, making it essential for researchers to develop fair and transparent AI systems. Additionally, the increasing reliance on AI for automation has sparked debates about job displacement, as machines replace human workers in various industries.

Also, I'm pregnant and very furious about it
"""
print(textrank_summary(text, 2))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

nltk.download("punkt")

def sentiment_summary(text, fraction=0.3):
    """
    Summarizes text while preserving sentiment.
    
    :param text: The input text to summarize
    :param fraction: The percentage of original text to keep (0.0 to 1.0)
    :return: A summarized version of the text
    """
    sentences = sent_tokenize(text)
    total_sentences = len(sentences)
    
    # Ensure at least 1 sentence is returned
    num_sentences = max(1, int(total_sentences * fraction))

    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(sentences)
    
    sentence_scores = np.array(tfidf_matrix.sum(axis=1)).flatten()
    
    # Get sentiment scores
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = [abs(analyzer.polarity_scores(s)['compound']) for s in sentences]
    
    # Combine importance & sentiment (weighted sum)
    combined_scores = sentence_scores * np.array(sentiment_scores)
    
    # Pick top sentences
    top_sentence_indices = combined_scores.argsort()[-num_sentences:][::-1]
    summary = [sentences[i] for i in sorted(top_sentence_indices)]
    
    return " ".join(summary)


In [ ]:
# text = """AI is amazing! It has transformed the world. However, some fear that AI will take away jobs and create inequality.
# Despite concerns, AI is advancing rapidly and helping in many fields like healthcare and automation."""
text = """
I really hate this place. I also went sleding the other day and had a blast. My wife and kids came along too. It has awful pizza, fries, and burgers. I also love the bartender though
"""
print(sentiment_summary(text, fraction=0.5))

In [42]:
import shap.maskers
import torch
import torch.nn as nn
import shap
import numpy as np
from transformers import BertModel, BertTokenizer

class BERTBusinessClassifier(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', hidden_dim=256, dropout=0.3):
        super(BERTBusinessClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.bert.config.hidden_size, hidden_dim)
        self.relu = nn.ReLU()
        self.out = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] token embedding
        x = self.dropout(pooled_output)
        x = self.fc(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.out(x)
        return self.sigmoid(x).squeeze()

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_text(text, max_length=512):
    encoding = tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
    return encoding['input_ids'].squeeze(), encoding['attention_mask'].squeeze()

# Example Usage
model = BERTBusinessClassifier()
text = "The service was terrible, and the food was overpriced. Never coming back!"
input_ids, attention_mask = preprocess_text(text)
input_ids = input_ids.unsqueeze(0)  # Add batch dimension
attention_mask = attention_mask.unsqueeze(0)

def f(text):
    input_ids, attention_mask = preprocess_text(text)
    return model(input_ids, attention_mask)

with torch.no_grad():
    prediction = model(input_ids, attention_mask)
print(f"Prediction (probability of closure): {prediction.item():.4f}")

wrapped_model = shap.models.TopKLM(model, tokenizer, k=100)
masker = shap.maskers.Text(tokenizer, mask_token="...", collapse_mask_token=True)



explainer = shap.Explainer(wrapped_model, masker)
shap_values = explainer([text])



Prediction (probability of closure): 0.5209


UnboundLocalError: cannot access local variable 'logits' where it is not associated with a value

In [26]:
import shap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import numpy as np

# Ensure initjs is called
shap.initjs()

# Prepare data
train_text, train_labels = shap.datasets.imdb()
vectorizer = TfidfVectorizer(max_features=10000)
train_vectors = vectorizer.fit_transform(train_text)

# Train model
model = LinearSVC().fit(train_vectors, train_labels)

# Create explainer
explainer = shap.LinearExplainer(model, train_vectors)

# Generate SHAP values
text_to_explain = "This movie was really great"
x = vectorizer.transform([text_to_explain])

# Convert sparse matrix to dense array
x_dense = x.toarray()

# Generate SHAP values for the dense array
shap_values = explainer.shap_values(x_dense)

# Visualize
shap.force_plot(explainer.expected_value, shap_values, x_dense)

In [2]:
import datasets
import numpy as np
import transformers

import shap

/home/abhay/Documents/Repo/abhayma1000/DS3010FinalProject/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = datasets.load_dataset("imdb", split="test")

# shorten the strings to fit into the pipeline model
short_data = [v[:500] for v in dataset["text"][:20]]

Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 199567.21 examples/s]


In [4]:
classifier = transformers.pipeline("sentiment-analysis", return_all_scores=True)
classifier(short_data[:2])

2025-03-03 12:36:04.724307: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-03 12:36:04.752484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741023364.785628    5301 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741023364.795235    5301 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 12:36:04.826401: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

[[{'label': 'NEGATIVE', 'score': 0.07581958174705505},
  {'label': 'POSITIVE', 'score': 0.9241803884506226}],
 [{'label': 'NEGATIVE', 'score': 0.018342511728405952},
  {'label': 'POSITIVE', 'score': 0.9816575050354004}]]

In [5]:
# define the explainer
explainer = shap.Explainer(classifier)

In [6]:
# explain the predictions of the pipeline on the first two samples
shap_values = explainer(short_data[:2])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
PartitionExplainer explainer: 3it [00:12, 12.02s/it]               


In [7]:
shap.plots.text(shap_values[:, :, "POSITIVE"])